In [1]:
import cv2
import uuid
import mediapipe as mp
import numpy as np

from tensorflow.keras.models import load_model


2021-10-18 11:19:51.245637: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2021-10-18 11:19:51.245662: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# load the model
model = load_model('Models/model1.h5')

2021-10-18 11:19:52.361241: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-18 11:19:52.362199: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2021-10-18 11:19:52.362480: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2021-10-18 11:19:52.362660: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 64)        1792      
_________________________________________________________________
activation (Activation)      (None, 98, 98, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 256)       147712    
_________________________________________________________________
activation_1 (Activation)    (None, 47, 47, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 128)       2

In [4]:
# Setting up variables
BLACK = (0, 0, 0)
RED = (0, 0, 255)
GREEN = (0, 255, 0)

font_simplex = cv2.FONT_HERSHEY_SIMPLEX
no_face_coords = (50, 250)
heading_coords = (150, 30)
thickness = 2
font_scale = 1
wearing_mask_message = "Thank you for wearing mask"
not_wearing_mask_message = "Please wear your mask"

In [5]:
# Define mediapipe Face detector
face_detection = mp.solutions.face_detection.FaceDetection()
def get_detection(frame):
    
    height, width, channel = frame.shape
    # Convert frame BGR to RGB colorspace
    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Detect results from the frame
    result = face_detection.process(imgRGB)
    # Extract data from result
    try:
        for count, detection in enumerate(result.detections):
            # print(detection)
            # Extract bounding box information
            box = detection.location_data.relative_bounding_box
            x, y, w, h = int(box.xmin*width), int(box.ymin * height), int(box.width*width), int(box.height*height)
          
    # If detection is not available then print No face
    except:
        cv2.putText(frame, "No face found...", no_face_coords, font_simplex, font_scale+1, BLACK, thickness+2, cv2.LINE_AA)
    
    return x, y, w, h

In [6]:
CATEGORIES = ['No Mask', 'Mask']
cap = cv2.VideoCapture(0)
while True:
    _, frame = cap.read()

    img = frame.copy()
    try:
        x, y, w, h = get_detection(frame)
        # print("frame details: ",x,y,w,h)
        crop_img = img[y:y+h, x:x+w]
        crop_img = cv2.resize(crop_img, (100, 100))
        crop_img = np.expand_dims(crop_img, axis=0)

        prediction = model.predict(crop_img)
        # print(prediction)

        index = np.argmax(prediction)
        res = CATEGORIES[index]
        # print(res, prediction, index)

        color = (0, 0, 0)
        if index == 0:
            # index 0 means no mask so red color
            color = RED
        else:
            # index 0 means no mask so green color
            color = GREEN
        
        # Draw a rectangle around the face and write the result on top of the rectangle
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, res, (x, y-10), font_simplex, 0.8, color, 2, cv2.LINE_AA)
            
    except Exception as e:
        # print(e)
        pass
    
    cv2.putText(frame, "Face Mask detection", heading_coords, font_simplex, font_scale, BLACK, thickness, cv2.LINE_AA)
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        exit()
        break

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2021-10-18 11:19:53.200991: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
